In [ ]:
import subprocess
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import List

In [ ]:
# Change working directory to the PathTracer
#os.chdir("C:/Users/mathias.gammelmark/Documents/repo/LightSourceImportanceSampling/PathTracer")
os.chdir("../PathTracer")

In [ ]:
def run(args:List[str]):
    process = subprocess.run(["../bin/Release-windows-x86_64/PathTracer.exe"] + args, stdout=None)

def arg_num_samples(n:int):
    return ["-n", str(n)]

def arg_load_obj(filepath:str):
    return ["-obj", filepath]

def arg_scene(files:List[str]):
    res = []
    for file in files:
        res = res + arg_load_obj(file)
    return res

def arg_cam_pos(x:float, y:float, z:float):
    return ["-pos", str(x), str(y), str(z)]

def arg_cam_rot(x:float, y:float, z:float):
    return ["-rot", str(x), str(y), str(z)]

def arg_output(filepath:str):
    return ["-out", filepath]

def arg_outdir(filepath:str):
    if (filepath.endswith("/")):
        return ["-outdir", filepath]
    else:
        return ["-outdir", filepath + "/"]



In [ ]:
figure_dpi = 100

def load_image_from_csv(filepath:str):
    data = pd.read_csv(filepath).to_numpy()
    #print(np.shape(data))
    img = np.zeros((512,512,3), dtype=np.float)

    for y in range(0,511):
        for x in range(0, 511):
            pixel = data[(511-y)*512 + x][:]
            img[y,x] = pixel

    return img

def exposure(img, exposure):
    # reinhard tone mapping
    return img / (img+exposure)

def gamma(img, gamma):
    return np.power(img, 1.0 / gamma)

def color_correct(img):
    return gamma(exposure(img, 1.0), 2.2)

def show_img(img):
    plt.figure(dpi=figure_dpi)
    plt.imshow(color_correct(img))
    plt.show()

In [ ]:
models = ["../Assets/Models/Helix.obj", "../Assets/Models/CornellBox.obj"]

args_scene = arg_scene(models)

In [ ]:
reference_count = 100000
def run_cornell_reference():
    models = arg_scene(["../Assets/Models/Helix.obj", "../Assets/Models/CornellBox.obj"])
    outdir = arg_outdir("../Test/Results/")
    args = models + outdir
    args += arg_num_samples(reference_count)
    args += ["-method", "naive"]
    args += ["-name", "cornell_ref"]
    run(args)

# Takes a very long time to compute
# run_cornell_reference()

In [ ]:
def run_cornell_scene():
    models = arg_scene(["../Assets/Models/Helix.obj", "../Assets/Models/CornellBox.obj"])
    outdir = arg_outdir("../Test/Results/")

    for samples in [1,2,4,8,16,32,64,128]:
        args = models + outdir
        args += ["-method", "lighttree"]
        args += arg_num_samples(samples)
        args += ["-name", "cornell_lighttree_samples_" + str(samples)]
        run(args)

run_cornell_scene()

In [ ]:
print("Running Pathtracer: Lighttree")
run(args_scene + arg_num_samples(100) + ["-method", "lighttree", "-name", "Lighttree"] + arg_outdir("../Test/Results/"))

In [ ]:
print("Running Pathtracer: Energy")
run(args_scene + arg_num_samples(100) + ["-method", "energy", "-name", "Energy"] + arg_outdir("../Test/Results/"))

In [ ]:
print("Running Pathtracer: Naive")
run(args_scene + arg_num_samples(100) + ["-method", "naive", "-name", "Naive"] + arg_outdir("../Test/Results/"))

In [ ]:
print("Loading Images")
resA = load_image_from_csv("../Test/Results/Lighttree.csv")
resB = load_image_from_csv("../Test/Results/Energy.csv")
resC = load_image_from_csv("../Test/Results/Naive.csv")

show_img(resA)
show_img(resB)
show_img(resC)

In [ ]:
res_ref = load_image_from_csv("../Test/Results/cornell_ref.csv")
show_img(res_ref)